## 1. Получение данных

Импортируем необходимые библиотеки, функции и структуры данных.

In [1]:
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Сохраним датасет в переменной data. Методом info() проверим, действительно ли не требуется предобработка данных о предоставленных услугах связи.

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')
display(data.head())

display(data.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

### Вывод
В предоставленном наборе данных имеется информация о 3214 объектах, которые характеризуются пятью признаками:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца.

Целевым признаком является - тариф пользователя, значение тарифа «Ультра» — 1, «Смарт» — 0.

Предобработка данных о поведении клиентов "Мегалайн" не требуется. В массиве отсутствуют пропущенные значения, признакам и целевому признаку соответсвует вещественный и целочисленный тип данных.

## 2. Разбиение данных на выборки

Разделим входные данные на обучающую, валидационную и тестовую выборки в соотношении: 3/1/1.

In [3]:
data_train, data_valid_test = train_test_split(data, test_size = 0.4, random_state = 12345)
data_valid, data_test = train_test_split(data_valid_test, test_size = 0.5, random_state = 12345)

Создадим переменные для признаков и целевого признака для всех выборок.

In [4]:
# переменные для обучающей выборки
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

# переменные для валидационной выборки
features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']

# переменные для тестовой выборки
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

Просмотрим размеры выборок

In [5]:
print(features_train.shape)
print(target_train.shape)

print(features_valid.shape)
print(target_valid.shape)

print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## 3. Исследование моделей

#### 3.1. Модель: решающее дерево

Рассмотрим модель классификации - решающее дерево. При обучении модели проварьируем гиперпараметр -  глубину дерева. Cохраним модель с лучшим значением метрики accuracy на валидационной выборке.

In [6]:
best_result = 0
best_max_depth = 0

for depth in range(1, 11):
    # построим модель с заданной глубиной
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # обучим модель на тренировочной выборке
    model.fit(features_train, target_train)
    
    # сохраним модель с наилучшим значением accuracy на валидационных данных
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_result = result
        best_max_depth = depth
        
print("Accuracy наилучшей исследованной модели решающего дерева на валидационной выборке:", best_result)
print('Максимальная глубина:', best_max_depth)

Accuracy наилучшей исследованной модели решающего дерева на валидационной выборке: 0.7853810264385692
Максимальная глубина: 3


#### 3.2. Модель: случайный лес

Рассмотрим модель - случайный лес. При обучении модели проварьируем 2 гиперпараметра:  глубину дерева и количество деревьев. Cохраним модель с лучшим значением метрики accuracy на валидационной выборке.

In [7]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0

for est in range(10, 51, 10):
    for depth in range(1, 11):
        # построим модель с заданным количеством деревьев и глубиной
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        # обучим модель на тренировочной выборке
        model.fit(features_train, target_train)  
        # посчитаем качество модели на валидационной выборке
        result = model.score(features_valid, target_valid)
        # сохраним модель с наилучшим значением accuracy на валидационных данных
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
            
print("Accuracy наилучшей исследованной модели случайного леса на валидационной выборке:", best_result)
print("Количество деревьев:", best_est)
print("Максимальная глубина:", depth)

Accuracy наилучшей исследованной модели случайного леса на валидационной выборке: 0.8087091757387247
Количество деревьев: 40
Максимальная глубина: 10


#### 3.3. Модель: логистическая регрессия

Обучим модель логистической регрессии на обучающей выборке и посчитаем её качество на валидационных данных.

In [8]:
model = LogisticRegression(random_state=12345) # построим модель
model.fit(features_train, target_train) # обучим модель на тренировочной выборке
result = model.score(features_valid, target_valid) # получим качество модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод
По результатам исследования трех моделей: решающее дерево, случайный лес и логистическая регрессия, наибольшим значением accuracy характеризуется модель случайного леса с количеством деревьев и максимальной глубиной - 40 и 10, соответственно. Для правильной оценки данной модели проверим ее на тестовой выборке.

## 4. Проверка модели на тестовой выборке

Зададим оптимальные характеристики модели случайного леса, установленные в части 3.2.

In [9]:
model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=10)
model.fit(features_train, target_train)

result = model.score(features_test, target_test)
print("Accuracy наилучшей исследованной модели случайного леса на тестовой выборке:", result)

Accuracy наилучшей исследованной модели случайного леса на тестовой выборке: 0.8009331259720062


## 5. (бонус) Проверка модели на адекватность

Для оценки исследованной модели случайного леса на вменяемость воспользуемся классификатором DummyClassifier, который генерирует случайные прогнозы. Выберем параметр классификатора strategy="most_frequent" - наиболее частотные значения.

In [10]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)

result = dummy_clf.score(features_test, target_test)
print('Accuracy составляет:', result)

Accuracy составляет: 0.6842923794712286


### Вывод:
Для решения задачи бинарной классификации для рекомендации подходящего тарифа для клиентов оператора связи - "Смарт" или "Ультра" - проведено исследование трех моделей(решающее дерево, случайный лес, логистическая регрессия) и построена модель случайного леса с уровнем **accuracy = ~0.809**. Пройдена проверка на адекватность полученной модели. Получено, что качество исследованной модели случайного леса выше по сравнению с классификатором DummyClassifier(accuracy = ~0.684).